In [1]:
import polars as pl
import pandas as pd
import os
import glob

In [62]:
# Path and file type
folder = '../data/parquet_files/train/'
file_type = 'train_credit_bureau_'

# Get file paths for file groups
files = {}
for group in ['a_1', 'a_2', 'b_1', 'b_2']:
    path = os.path.join(folder, file_type + group + '*')
    file_group = glob.glob(path)
    files[group] = file_group

## Train Credit Bureau A1 Columns

In [113]:
# Check a1 file
df = pl.read_parquet(files['a_1'][0])
print(f'Shape: {df.shape}')
print(f"Number of Unique Case IDs: {df['case_id'].n_unique()}")

Shape: (4108212, 79)
Number of Unique Case IDs: 335275


In [120]:
# Column dtypes
dtype_counts = pd.Series(df.dtypes).value_counts()
print(dtype_counts)

# Date columns
date_cols = [ df.columns[i] for i in range(len(df.columns)) if (df.columns[i].__contains__('dat')) and (df.dtypes[i] == pl.String) ]

# Categorical columns
cat_cols = [ df.columns[i] for i in range(len(df.columns)) if (df.columns[i] not in date_cols) and (df.dtypes[i] == pl.String) ]

# Numerical columns
ignore_cols = ['case_id', 'num_group1', 'num_group2']
num_cols = [ 
    df.columns[i] for i in range(len(df.columns)) 
    if (df.columns[i] not in date_cols) and (df.columns[i] not in cat_cols) and (df.columns[i] not in ignore_cols)
]

Float64    54
String     23
Int64       2
Name: count, dtype: int64


In [121]:
# Convert date columns
df_date = df[['case_id'] + date_cols].with_columns([ pl.col(col).str.to_date() for col in date_cols ])

# One-hot categories
df_dummies = df[['case_id'] + cat_cols].to_dummies(cat_cols)

In [122]:
# Date aggs
date_aggs = [ pl.min(col).name.suffix('_min') for col in date_cols ] +\
            [ pl.max(col).name.suffix('_max') for col in date_cols ] +\
            [ pl.n_unique(col).name.suffix('_distinct') for col in date_cols]
df_date_grouped = df_date.group_by('case_id').agg(date_aggs)

# One-hot aggs
dummy_cols = [ col for col in df_dummies.columns if col != 'case_id']
dummies_aggs = [ pl.sum(col).name.suffix('_sum') for col in dummy_cols ]
df_dummies_grouped = df_dummies.group_by('case_id').agg(dummies_aggs)

# Numerical aggs
num_aggs = [ pl.min(col).name.suffix('_min') for col in num_cols ] +\
           [ pl.max(col).name.suffix('_max') for col in num_cols ] +\
           [ pl.mean(col).name.suffix('_mean') for col in num_cols ] +\
           [ pl.median(col).name.suffix('_median') for col in num_cols ] +\
           [ pl.sum(col).name.suffix('_sum') for col in num_cols ]
df_num_grouped = df.group_by('case_id').agg(num_aggs)

In [123]:
# Join DataFrames
df_joined = df_num_grouped.join(df_date_grouped, on='case_id')
df_joined = df_joined.join(df_dummies_grouped, on='case_id')

In [124]:
df_joined.head()

case_id,annualeffectiverate_199L_min,annualeffectiverate_63L_min,contractsum_5085717L_min,credlmt_230A_min,credlmt_935A_min,debtoutstand_525A_min,debtoverdue_47A_min,dpdmax_139P_min,dpdmax_757P_min,dpdmaxdatemonth_442T_min,dpdmaxdatemonth_89T_min,dpdmaxdateyear_596T_min,dpdmaxdateyear_896T_min,instlamount_768A_min,instlamount_852A_min,interestrate_508L_min,monthlyinstlamount_332A_min,monthlyinstlamount_674A_min,nominalrate_281L_min,nominalrate_498L_min,numberofcontrsvalue_258L_min,numberofcontrsvalue_358L_min,numberofinstls_229L_min,numberofinstls_320L_min,numberofoutstandinstls_520L_min,numberofoutstandinstls_59L_min,numberofoverdueinstlmax_1039L_min,numberofoverdueinstlmax_1151L_min,numberofoverdueinstls_725L_min,numberofoverdueinstls_834L_min,outstandingamount_354A_min,outstandingamount_362A_min,overdueamount_31A_min,overdueamount_659A_min,overdueamountmax2_14A_min,overdueamountmax2_398A_min,…,purposeofcred_426M_9e302002_sum,purposeofcred_426M_P188_162_121_sum,purposeofcred_426M_a55475b1_sum,purposeofcred_426M_e19fdece_sum,purposeofcred_426M_e8f3b178_sum,purposeofcred_874M_27b6de28_sum,purposeofcred_874M_28bfa260_sum,purposeofcred_874M_44164129_sum,purposeofcred_874M_5065c2b8_sum,purposeofcred_874M_5d1b0cdd_sum,purposeofcred_874M_60c73645_sum,purposeofcred_874M_7a7d6960_sum,purposeofcred_874M_7ff464db_sum,purposeofcred_874M_8193a6ce_sum,purposeofcred_874M_89ccf2a3_sum,purposeofcred_874M_96a8fdfe_sum,purposeofcred_874M_P188_162_121_sum,purposeofcred_874M_a55475b1_sum,purposeofcred_874M_d11871e7_sum,purposeofcred_874M_d9ae1a0e_sum,purposeofcred_874M_e19fdece_sum,purposeofcred_874M_e8f3b178_sum,purposeofcred_874M_ee7d1eb8_sum,subjectrole_182M_0c42a10e_sum,subjectrole_182M_15f04f45_sum,subjectrole_182M_P28_48_88_sum,subjectrole_182M_a55475b1_sum,subjectrole_182M_ab3c25cf_sum,subjectrole_182M_be4fd70b_sum,subjectrole_182M_daf49a8a_sum,subjectrole_93M_0c42a10e_sum,subjectrole_93M_15f04f45_sum,subjectrole_93M_71ddaa88_sum,subjectrole_93M_a55475b1_sum,subjectrole_93M_ab3c25cf_sum,subjectrole_93M_be4fd70b_sum,subjectrole_93M_daf49a8a_sum
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1383971,null,null,null,null,200000.0,140271.67,18627.041,1.0,0.0,5.0,6.0,2019.0,2018.0,21100.293,null,null,21100.293,17686.6,null,0.12,1.0,1.0,18.0,null,0.0,null,2.0,1.0,2.0,0.0,0.0,null,0.0,18627.041,18627.041,17682.0,…,0,0,10,0,0,0,0,0,0,0,1,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0,10,1,0,0
701157,8.11,5.5,null,4000.0,null,1115893.1,0.0,0.0,0.0,1.0,6.0,2017.0,2010.0,null,400.0,null,6955.0,0.0,5.0,8.0,2.0,1.0,57.0,72.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,47877.715,0.0,0.0,0.0,0.0,…,0,0,9,0,0,0,0,0,1,0,1,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0,9,1,1,0
1399149,29.8,null,null,26998.0,20000.0,216750.98,0.0,0.0,0.0,11.0,1.0,2017.0,2017.0,6886.0,473.03198,null,3958.4001,473.03198,29.4,23.0,4.0,2.0,36.0,24.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,65717.26,0.0,0.0,0.0,0.0,…,0,0,7,0,0,0,0,0,0,0,2,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0,10,1,0,0
1270453,null,26.4,null,null,null,47993.332,0.0,0.0,null,null,1.0,2018.0,null,null,null,null,1778.0,null,23.56,null,2.0,10.0,null,10.0,null,8.0,0.0,null,0.0,null,null,13043.526,null,0.0,0.0,null,…,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,11,1,0,0,0,0,0,11,1,0,0
105283,null,null,null,null,10150.0,0.0,0.0,1.0,null,null,12.0,2018.0,null,300.514,null,null,300.514,null,39.0,null,1.0,9.0,null,null,null,null,1.0,null,0.0,null,null,null,null,0.0,610.2,null,…,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0,10,1,0,0


In [143]:
def group_file_data(
    df: pl.DataFrame, 
    num_cols: list[str] = [], 
    date_cols: list[str] = [], 
    cat_cols: list[str] = []
) -> pl.DataFrame:
    '''
    Function to group numerical, date, and categorical columns

    Parameters:
    -----------
    df : Polars DataFrame
    num_cols : List of numerical column names (remember to drop num_group columns)
    date_cols : List of date column names
    cat_cols : List of categorical column names (becomes dummies)
    '''
    
    # Convert date columns
    df_date = df[['case_id'] + date_cols].with_columns([ pl.col(col).str.to_date() for col in date_cols ])

    # One-hot categories
    df_dummies = df[['case_id'] + cat_cols].to_dummies(cat_cols)

    # Num DataFrame
    df_num = df[['case_id'] + num_cols]

    # Date aggs
    date_aggs = [ pl.min(col).name.suffix('_min') for col in date_cols ] +\
                [ pl.max(col).name.suffix('_max') for col in date_cols ] +\
                [ pl.n_unique(col).name.suffix('_distinct') for col in date_cols]
    df_date_grouped = df_date.group_by('case_id').agg(date_aggs)

    # One-hot aggs
    dummy_cols = [ col for col in df_dummies.columns if col != 'case_id']
    dummies_aggs = [ pl.sum(col).name.suffix('_sum') for col in dummy_cols ]
    df_dummies_grouped = df_dummies.group_by('case_id').agg(dummies_aggs)

    # Numerical aggs
    num_aggs = [ pl.min(col).name.suffix('_min') for col in num_cols ] +\
            [ pl.max(col).name.suffix('_max') for col in num_cols ] +\
            [ pl.mean(col).name.suffix('_mean') for col in num_cols ] +\
            [ pl.median(col).name.suffix('_median') for col in num_cols ] +\
            [ pl.sum(col).name.suffix('_sum') for col in num_cols ]
    df_num_grouped = df_num.group_by('case_id').agg(num_aggs)

    # Join DataFrames
    df_joined = df_num_grouped.join(df_date_grouped, on='case_id')
    df_joined = df_joined.join(df_dummies_grouped, on='case_id')

    return df_joined

In [144]:
# Concat a1 files
df_file = pl.DataFrame()
for f in files['a_1']:
    temp_df = pl.read_parquet(f)
    df_file = pl.concat([df_file, temp_df])

# Group data
df = group_file_data(df_file, num_cols, date_cols, cat_cols)

In [145]:
df.write_parquet('../data/train_credit_bureau_a_1_grouped.parquet')

## Train Credit Bureau A2 Columns

In [146]:
df_a2 = pl.read_parquet(files['a_2'][8])
df_a2.shape

(5296031, 19)

In [147]:
df_a2.head()

case_id,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,num_group1,num_group2,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T,subjectroles_name_541M,subjectroles_name_838M
i64,str,str,f64,f64,str,str,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
388,"""8fd95e4b""","""a55475b1""",0.0,null,"""a55475b1""","""c7a5ad39""",0,0,null,null,2.0,null,null,null,2018.0,null,"""a55475b1""","""ab3c25cf"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,1,null,null,3.0,null,null,null,2018.0,null,"""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,2,null,null,4.0,null,null,null,2018.0,null,"""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,3,null,null,5.0,null,null,null,2018.0,null,"""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,4,null,null,6.0,null,null,null,2018.0,null,"""a55475b1""","""a55475b1"""


In [148]:
# Besides some messy payment stuff (will deal with later), straight forward groups
a2_date_cols = []
a2_cat_cols = [ df_a2.columns[i] for i in range(len(df_a2.columns)) if (df_a2.columns[i] not in a2_date_cols) and (df_a2.dtypes[i] == pl.String) ]
a2_num_cols = [ 
    df_a2.columns[i] for i in range(len(df_a2.columns)) 
    if (df_a2.columns[i] not in a2_date_cols) and (df_a2.columns[i] not in a2_cat_cols) and (df_a2.columns[i] not in ignore_cols)
]

In [149]:
# Concat a2 files
df_a2_file = pl.DataFrame()
for f in files['a_2']:
    temp_df = pl.read_parquet(f)
    df_a2_file = pl.concat([df_a2_file, temp_df])

# Group data
df_a2 = group_file_data(df_a2_file, a2_num_cols, a2_date_cols, a2_cat_cols)

In [150]:
df_a2.write_parquet('../data/train_credit_bureau_a_2_grouped.parquet')

## Train Credit Bureau B1 Columns

In [151]:
df_b1 = pl.read_parquet(files['b_1'][0])
df_b1.shape

(85791, 45)

In [152]:
df_b1.head()

case_id,amount_1115A,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,contracttype_653M,credlmt_1052A,credlmt_228A,credlmt_3940954A,credor_3940957M,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,debtvalue_227A,dpd_550P,dpd_733P,dpdmax_851P,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,installmentamount_644A,installmentamount_833A,instlamount_892A,interesteffectiverate_369L,interestrateyearly_538L,lastupdate_260D,maxdebtpduevalodued_3940955A,num_group1,numberofinstls_810L,overdueamountmax_950A,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,periodicityofpmts_997L,periodicityofpmts_997M,pmtdaysoverdue_1135P,pmtmethod_731M,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A
i64,f64,str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,i64,f64,f64,f64,f64,str,str,f64,str,f64,str,f64,f64,f64,str,str,f64,f64
467,null,"""ea6782cc""","""2011-06-15""","""2031-06-13""","""7241344e""","""724be82a""",3e6,10000.0,3e6,"""P164_34_168""",2.0,1.0,null,null,0.0,0.0,null,null,null,0.0,0.0,null,null,null,"""2019-01-20""",null,0,null,null,null,null,null,"""a55475b1""",null,"""a55475b1""",null,"""96a8fdfe""",0.0,0.0,null,"""fa4f56f1""","""ab3c25cf""",3e6,10000.0
467,null,"""ea6782cc""","""2019-01-04""","""2021-08-04""","""7241344e""","""724be82a""",null,null,130365.0,"""P164_34_168""",1.0,2.0,null,null,0.0,0.0,null,null,null,0.0,26571.969,null,null,null,"""2019-01-20""",null,1,null,null,null,null,null,"""a55475b1""",null,"""a55475b1""",null,"""96a8fdfe""",null,null,null,"""ab3c25cf""","""ab3c25cf""",78000.0,960000.0
467,78000.0,"""ea6782cc""","""2016-10-25""","""2019-10-25""","""7241344e""","""4257cbed""",null,null,null,"""c5a72b57""",null,null,0.0,26571.969,null,null,0.0,11.0,2016.0,null,null,2898.76,null,null,"""2019-01-10""",0.0,2,36.0,0.0,11.0,2016.0,null,"""a0b598e4""",0.0,"""e914c86c""",10.0,"""96a8fdfe""",null,null,null,"""a55475b1""","""a55475b1""",null,null
1445,null,"""ea6782cc""","""2015-01-30""","""2021-01-30""","""7241344e""","""1c9c5356""",400000.0,100000.0,74000.0,"""b619fa46""",2.0,5.0,0.0,null,0.0,0.0,200418.0,1.0,2018.0,0.0,0.0,null,null,null,"""2019-01-19""",0.4,0,null,1.4,2.0,2018.0,null,"""a55475b1""",0.0,"""a55475b1""",null,"""60c73645""",0.0,0.0,73044.18,"""daf49a8a""","""ab3c25cf""",400000.0,100000.0
1445,null,"""01f63ac8""","""2014-09-12""","""2021-09-12""","""7241344e""","""724be82a""",null,null,400000.0,"""74bd67a8""",3.0,17.0,null,null,0.0,0.0,null,null,null,0.0,209617.77,null,null,null,"""2019-01-13""",null,1,null,null,null,null,null,"""a55475b1""",null,"""a55475b1""",null,"""96a8fdfe""",null,null,null,"""ab3c25cf""","""ab3c25cf""",396800.62,184587.8


In [153]:
b1_date_cols = [ df_b1.columns[i] for i in range(len(df_b1.columns)) if (df_b1.columns[i].__contains__('dat')) and (df_b1.dtypes[i] == pl.String) ]
b1_cat_cols = [ df_b1.columns[i] for i in range(len(df_b1.columns)) if (df_b1.columns[i] not in b1_date_cols) and (df_b1.dtypes[i] == pl.String) ]
b1_num_cols = [ 
    df_b1.columns[i] for i in range(len(df_b1.columns)) 
    if (df_b1.columns[i] not in b1_date_cols) and (df_b1.columns[i] not in b1_cat_cols) and (df_b1.columns[i] not in ignore_cols)
]

In [154]:
# Group data
df_b1 = group_file_data(df_b1, b1_num_cols, b1_date_cols, b1_cat_cols)

df_b1.write_parquet('../data/train_credit_bureau_b_1_grouped.parquet')

## Train Credit Bureau B2 Columns

In [155]:
df_b2 = pl.read_parquet(files['b_2'][0])
df_b2.shape

(1286755, 6)

In [156]:
df_b2.head()

case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
i64,i64,i64,str,f64,f64
467,2,19,"""2018-05-15""",0.0,0.0
467,2,25,"""2018-11-15""",0.0,0.0
467,2,18,"""2018-04-15""",0.0,0.0
467,2,0,"""2016-10-15""",0.0,0.0
467,2,6,"""2017-04-15""",0.0,0.0


In [157]:
# Again depth 2 files are a bit more messy and we will deal with the messiness later
b2_date_cols = [ df_b2.columns[i] for i in range(len(df_b2.columns)) if (df_b2.columns[i].__contains__('dat')) and (df_b2.dtypes[i] == pl.String) ]
b2_cat_cols = [ df_b2.columns[i] for i in range(len(df_b2.columns)) if (df_b2.columns[i] not in b2_date_cols) and (df_b2.dtypes[i] == pl.String) ]
b2_num_cols = [ 
    df_b2.columns[i] for i in range(len(df_b2.columns)) 
    if (df_b2.columns[i] not in b2_date_cols) and (df_b2.columns[i] not in b2_cat_cols) and (df_b2.columns[i] not in ignore_cols)
]

In [158]:
# Group data
df_b2 = group_file_data(df_b2, b2_num_cols, b2_date_cols, b2_cat_cols)

df_b2.write_parquet('../data/train_credit_bureau_b_2_grouped.parquet')